# LLM을 활용한 ChatBot 앱 구축
LLM을 기반으로 챗봇을 설계하고 구현하느 방법에 대해 기술한다.

챗봇은 아래 요소를 포함한다.
- `Chat Models` : 챗봇 인터페이스는 텍스트 대신 메시지를 기반으로 한다.
- `Prompt Templates` : 기본 메시지, 사용자 입력, 대화 기록 및 추가로 검색 된 컨텍스트를 결합하여 프롬프트를 구성하는 과정을 단순화 한다.
- `Chat History` : 챗봇이 이전 대화 내역을 기억하여 후속 질문 응답시 이를 참고한다.
- `LangSmith` : 애플리케이션 디버깅 및 흐름 추적을 위해 사용한다.

## 라이브러리 설치
아래 명령어를 통해 LangChain을 설치한다.

In [ ]:
!pip install langchain langchain-openai

## LangSmith 연동
흐름 분석을 위해 아래 변수 설정으로 LangSmith와 연동한다.

In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "Insert Your LangSmith API Key"

## LLM 정의
LLM을 사용하기 위해 라이브러리를 호출하고 인스턴스를 생성한다.

In [ ]:
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "Insert Your OpenAI API Key"
model = ChatOpenAI(model="gpt-4o")

## LLM 호출
`.invoke` 메서드에 메시지 목록을 전달하여 모델을 간단히 호출할 수 있다.

In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

모델 자체에는 상태 개념이 없기때문에 후속 질문을 하면 질문에 답할 수 없다.

In [ ]:
model.invoke([HumanMessage(content="What's my name?")])

모델은 이전 대화 내역이 없기에 질문을 답할 수 없다.

챗봇과의 대화가 자연스러워지려면 전체 대화 기록을 모델에 전달해야한다.

In [ ]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hi! I'm Bob"),
    AIMessage(content="Hello Bob! How can I assist you today?"),
    HumanMessage(content="What's my name?"),
])

## 챗봇 메모리 기능 구현
Message History 클래스를 사용하여 모델을 래핑하고 상태를 유지할 수 있다.

이는 모델의 입력과 출력을 추적하고 일부 데이터 저장소에 저장한다.

먼저 추가 라이브러리를 설치한다.

In [ ]:
!pip install langchain_community

Message History 클래스를 가져와서 체인을 설정한다.

체인은 모델을 래핑하고 메시지 히스토리를 추가한다.

`get_session_history`는 `session_id`를 입력으로 받아 Message History 객체를 반환해야한다.

`session_id`는 별도의 대화를 구분하는데 사용되며, 새로운 체인을 호출할 때 구성의 일부로 전달해야 한다.

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

모델에 메시지를 전달할 때 포함할 `session_id`를 위해 `config`를 생성한다.

In [ ]:
config = {"configurable": {"session_id": "abc2"}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

이제 이름을 기억하는지 확인해보자.

In [ ]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

## 프롬프트 템플릿
사용자 정의 지침이 포함된 시스템 메시지를 추가하여 의도된 형태의 답변이 올 수 있게 설정할 수 있다.

프롬프트 템플릿 설정을 위해 `ChatPromptTemplate`을 생성하고 `MessagesPlaceholder`를 사용하여 모든 메시지를 전달한다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

프롬프트 템플릿 설정 시 입력 유형이 약간 변경된다.

메시지 목록을 전달하는 대신 메시지 목록이 포함 된 `messages` 키가 있는 딕셔너리를 전달한다.

In [ ]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

기억 기능 구현을 위해 이전 Messages History 클래스로 래핑 후 메시지를 발송한다.

In [ ]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [ ]:
config = {"configurable": {"session_id": "abc5"}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

## 프롬프트 심화
프롬프트에 변수를 넣어 복잡하게 구현할 수 있다.

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [ ]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Korean"}
)

response.content

복잡해진 체인을 메시지 히스토리 클래스에 래핑한다.

In [ ]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [ ]:
config = {"configurable": {"session_id": "abc11"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Korean"},
    config=config,
)

response.content


In [ ]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Korean"},
    config=config,
)

response.content

## 메시지 크기 관리를 위한 대화 히스토리 관리

히스토리를 관리하지 않으면 메시지 목록이 무한히 증가하여 LLM의 컨텍스트 창을 초과할 수 있다.

따라서 전달하는 메시지 크기를 제한하는 단계를 추가하는 것이 중요하다.

이를 위해 **프롬프트 템플릿 구성 전에 이전 메시지를 히스토리에서 로드한 후 추가**해야한다.

아래 코드는 프롬프트 앞에 메시지를 적절히 수정하는 단계를 추가하고 새 체인을 생성한다.

마지막 `k`개의 메시지를 선택하도록 구현한 코드이다.

In [ ]:
from langchain_core.runnables import RunnablePassthrough

def filter_messages(messages, k=10):
    return messages[-k:]

chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

위와 같이 설정하면 10개 이상의 메시지 목록을 생성하면 초기 메시지의 정보를 더 이상 기억하지 못하게 된다.

In [ ]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [ ]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "Korean",
    }
)
response.content

In [ ]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
        "language": "Korean",
    }
)
response.content

## 스트리밍

LLM이 응답 메시지를 생성하는데 시간이 소요되기 때문에 사용자는 불편함을 느끼게 된다.

이를 최소화하기 위해 `.stream` 메서드를 사용하여 스트리밍을 구현할 수 있다.

In [ ]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "Korean",
    },
    config=config,
):
    print(r.content, end="")